In [1]:
#import
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
df = pd.read_csv('heart_out.csv')

In [3]:
df.head()

,Unnamed: 0,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,ca,...,restecg_0,restecg_1,restecg_2,slope_0,slope_1,slope_2,thal_0,thal_1,thal_2,thal_3
0,0,63,1,145,233,1,150,0,2.3,0,...,1,0,0,1,0,0,0,1,0,0
1,1,37,1,130,250,0,187,0,3.5,0,...,0,1,0,1,0,0,0,0,1,0
2,2,41,0,130,204,0,172,0,1.4,0,...,1,0,0,0,0,1,0,0,1,0
3,3,56,1,120,236,0,178,0,0.8,0,...,0,1,0,0,0,1,0,0,1,0
4,4,57,0,120,354,0,163,1,0.6,0,...,0,1,0,0,0,1,0,0,1,0


In [4]:
X = df.drop(columns = ['target'])
y = df['target']
print(y.head())
print(X.head())


0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64
   Unnamed: 0  age  sex  trestbps  chol  fbs  thalach  exang  oldpeak  ca  \
0           0   63    1       145   233    1      150      0      2.3   0   
1           1   37    1       130   250    0      187      0      3.5   0   
2           2   41    0       130   204    0      172      0      1.4   0   
3           3   56    1       120   236    0      178      0      0.8   0   
4           4   57    0       120   354    0      163      1      0.6   0   

   ...  restecg_0  restecg_1  restecg_2  slope_0  slope_1  slope_2  thal_0  \
0  ...          1          0          0        1        0        0       0   
1  ...          0          1          0        1        0        0       0   
2  ...          1          0          0        0        0        1       0   
3  ...          0          1          0        0        0        1       0   
4  ...          0          1          0        0        0        1       0   

   tha

In [86]:
gnb = GaussianNB()
kf = KFold(n_splits = 3, shuffle=True, random_state=22)
for train, test in kf.split(X):
    gnb.fit(X.iloc[train], y.iloc[train])
    print(gnb.score(X.iloc[test], y.iloc[test]))

0.8316831683168316
0.900990099009901
0.8811881188118812


In [85]:
forest_size = [1, 5, 10, 50, 100]
kf = KFold(n_splits = 3, shuffle=True, random_state=22)
for train, test in kf.split(X):
    for count in forest_size:
        rand_forest = RandomForestClassifier(criterion='gini', n_estimators=count, max_depth=3, random_state=0, n_jobs=-1)
        rand_forest.fit(X.iloc[train], y.iloc[train])
        print(rand_forest.score(X.iloc[test], y.iloc[test]))
    print()
    print()

0.7128712871287128
0.900990099009901
0.9702970297029703
0.9900990099009901
0.9900990099009901


0.7524752475247525
0.9108910891089109
0.9603960396039604
0.9702970297029703
1.0


0.801980198019802
0.9504950495049505
0.9801980198019802
0.9900990099009901
0.9801980198019802




In [92]:

kf = KFold(n_splits = 3, shuffle=True, random_state=22)
for train, test in kf.split(X):
    for knnCount in range(1,20):
        knn = KNeighborsClassifier(n_neighbors = knnCount, p = 1, metric = 'minkowski') # manhattan dist here
        knn.fit(X.iloc[train], y.iloc[train])
        print(knn.score(X.iloc[test], y.iloc[test]))
    print()
    print()


0.9702970297029703
0.9801980198019802
0.9900990099009901
0.9900990099009901
0.9900990099009901
0.9801980198019802
0.9801980198019802
0.9702970297029703
0.9801980198019802
0.9702970297029703
0.9801980198019802
0.9702970297029703
0.9801980198019802
0.9900990099009901
0.9900990099009901
0.9900990099009901
1.0
1.0
1.0


0.9603960396039604
0.9702970297029703
0.9603960396039604
0.9801980198019802
0.9504950495049505
0.9504950495049505
0.9405940594059405
0.9504950495049505
0.9405940594059405
0.9504950495049505
0.9702970297029703
0.9900990099009901
0.9801980198019802
1.0
0.9900990099009901
1.0
0.9900990099009901
0.9900990099009901
0.9900990099009901


0.9801980198019802
0.9801980198019802
1.0
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9801980198019802
0.9900990099009901


